In [ ]:
# !pip install saliency
# !pip install tensorflow

In [ ]:
# from google.colab import drive
# drive._mount('/content/gdrive')

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from keras.applications.inception_v3 import preprocess_input as inception_preproc
from tensorflow.keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgba2rgb
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import saliency
import os
import tensorflow.compat.v1 as tf
import tensorflow.python.keras.backend as K
# tf.disable_v2_behavior()

In [ ]:
# !pip list -v | grep saliency

In [ ]:
from seca.utils import load_image, save_image

In [ ]:
def setup_model(graph, weights: str = "imagenet"):
    """[summary]

    Parameters
    ----------
    graph : [type]
        [description]
    weights : [type], optional
        [description], by default "imagenet"

    Returns
    -------
    [type]
        [description]
    """

    model = InceptionV3(weights=weights)
    logits = graph.get_tensor_by_name('predictions/Softmax:0')
    neuron_selector = tf.placeholder(tf.int32)  # Used to select the logit of the prediction
    y = logits[0][neuron_selector]  # logit of prediction
    prediction = tf.argmax(logits, 1)
    images = graph.get_tensor_by_name('input_1:0') 

    return model, y, neuron_selector, prediction, images


def verify_images(path_images: str):
    """Verify images in dataset

    Parameters
    ----------
    path_images : str
        directory with images dataset
    """
    # TODO: Check file extensions
    # TODO: Check file size > 0

    # Print image names
    print(([name for name in os.listdir(path_images)]))


def upload_data_to_model(path_images: str, sample_size: int = None, target_size: tuple = (299, 299), batch_size: int = 1):
    """[summary]

    Parameters
    ----------
    path_images : [type]
        [description]
    sample_size : [type], optional
        [description], by default None
    target_size : [type], optional
        [description], by default (299, 299)
    batch_size : [type], optional
        [description], by default 1

    Returns
    -------
    selected_images : list[str]
        List of selected image paths

    Raises
    ------
    ValueError
        [description]
    ValueError
        [description]
    """
    
    test_datagen = ImageDataGenerator(preprocessing_function=inception_preproc)
    test_generator = test_datagen.flow_from_directory(path_images, target_size=target_size, batch_size=batch_size, class_mode='categorical', shuffle=False)
    
    # Select random sample if num_of_images is smaller than dataset
    num_images = len(test_generator.filenames) if not sample_size else sample_size

    if num_images > len(test_generator.filenames):
        print(len(test_generator.filenames))
        raise ValueError('The number of annotations cannot be higher than the number of available images.')
    elif num_images == 0:
        raise ValueError('The number of annotations needs to be greater than zero.')
    
    image_selection = np.random.choice(len(test_generator.filenames), num_images)
    selected_images = np.array(test_generator.filenames)[image_selection]
    
    return selected_images


def create_output_folder(path: str):
    """[summary]

    Parameters
    ----------
    path_output : str
        [description]
    image_name : str
        [description]
    """
        
    if not os.path.exists(path):
        os.makedirs(path)
    

def export_heatmap(image, smoothgrad_mask_grayscale, path_save: str, colormap: str = "inferno"):
    """[summary]

    Parameters
    ----------
    image : [type]
        [description]
    path_image : [type]
        [description]
    colormap : [type], optional
        [description], by default "inferno"
    """

    cm = plt.get_cmap(colormap)
    colored_heatmap = cm(smoothgrad_mask_grayscale)  # RGBA (A contains colormap) -> convert o RGB via rgba2rgb
    image_overlay = 0.5 * (image/255) + 0.5 * rgba2rgb(colored_heatmap)  # img1*alpha + img2*(1-alpha)
    plt.imsave(path_save, image_overlay)


In [ ]:
# Ensure proper handling of https requests
# Issue and solution are described here: https://github.com/tensorflow/tensorflow/issues/33285

import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
# Set parameters

path_images = "../example_data copy/test"
path_output = "../../output"
num_of_images = None

In [ ]:
# Initialize session
sess = K.get_session()
graph = sess.graph

# Check dataset
# verify_images(path_images)
selected_images = upload_data_to_model(path_images, num_of_images)

with graph.as_default():  # registers graph as default graph. Operations will be added to the graph
            
    model, y, neuron_selector, prediction, images = setup_model(graph, weights='imagenet')

    # Construct the saliency object.
    gradient_saliency = saliency.tf1.GradientSaliency(graph, sess, y, images)
    
    for img in tqdm(selected_images):
        
        path_image = Path(img)

        # Create the folder   
        output_folder = os.path.join(path_output, path_image.parents[0])
        create_output_folder(output_folder)

        # Skip if heatmap is already extracted
        path_export_heatmap = os.path.join(output_folder, f'{path_image.stem}_heatmap.jpg' )
        if not path_export_heatmap:
            continue

        # Load image
        image = load_image(os.path.join(path_images, path_image))
        im = inception_preproc(image)

        # Predict label
        y_pred = sess.run(prediction, feed_dict={images: [im]})[0]

        # Compute the vanilla mask and the smoothed mask.
        smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(im, stdev_spread=.05, nsamples=10, feed_dict={neuron_selector: y_pred})

        # Call the visualization methods to convert the 3D tensors to 2D grayscale.
        smoothgrad_mask_grayscale = saliency.tf1.VisualizeImageGrayscale(smoothgrad_mask_3d)        

        # Save images and heatmaps
        save_image(image, output_folder, path_image.name)
        export_heatmap(image, smoothgrad_mask_grayscale, path_export_heatmap)

